In [ ]:
# Gated Recurrent Unit - LSTM
import tensorflow as tf
import numpy as np
import pandas as pd
import re
import string
from string import digits
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)

<ipython-input-45-a3b3fedea4e5>:10: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [ ]:
lines = pd.read_csv('/content/drive/MyDrive/Sem_8-G21/Datasets/gu-en.tsv',sep='\t',names=['english_sentence', 'guj_sentence'])
lines = lines[:1000]
lines.head()

,english_sentence,guj_sentence
0,"The Union Cabinet chaired by Prime Minister Shri Narendra Modi has approved the proposal for closure of Autonomous Bodies, namely, Rashtriya Arogya Nidhi (RAN) and Jansankhya Sthirata Kosh (JSK) and the functions are proposed to be vested in Department of Health & Family Welfare (DoHFW).",પ્રધાનમંત્રી શ્રી નરેન્દ્ર મોદીની અધ્યક્ષતામાં મંત્રીમંડળે સ્વાસ્થ્ય અને પરિવાર કલ્યાણ વિભાગ અંતર્ગત સ્વાયત્ત સંસ્થાઓને તર્કસંગત બનાવવાની મંજૂરી આપી છે.
1,The rationalization of Autonomous Bodies under Department of Health & Family Welfare will involve inter-ministerial consultations and review of existing bye laws of these bodies.,આ સંસ્થાઓમાં રાષ્ટ્રીય આરોગ્ય નિધિ (આરએએન) અને જનસંખ્યા સ્થિરતા કોષ (જેએસકે) સામેલ છે.
2,"The time frame for implementation is one year,",તેનાં કામકાજને સ્વાસ્થ્ય અને પરિવાર કલ્યાણ વિભાગ અંતર્ગત રાખવાનો પ્રસ્તાવ પણ છે.
3,Rashtriya Arogya Nidhi (RAN) was set up as a registered society to provide financial medical assistance to poor patients receiving treatment in designated central government hospitals.,"રાષ્ટ્રીય આરોગ્ય નિધિની રચના એક નોંધાયેલી સોસાયટી સ્વરૂપે કરવામાં આવી હતી, જેથી કેન્દ્રીય સરકારી હોસ્પિટલોમાં સારવાર કરાવતાં ગરીબ દર્દીઓને નાણાકીય ચિકિત્સા સહાયતા આપી શકાશે."
4,An advance is placed with the Medical Superintendents of such hospitals who then provide assistance on a case to case basis.,"આ હોસ્પિટલનાં તબીબી નિરીક્ષકોને આગોતરૂ ભંડોળ આપવામાં આવશે, જે દરેક કેસને જોઈને સહાયતા પ્રદાન કરશે."


In [ ]:
# Lowercase all characters
lines['english_sentence']=lines['english_sentence'].apply(lambda x: str(x))
lines['guj_sentence']=lines['guj_sentence'].apply(lambda x: str(x))
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.lower())
lines['guj_sentence']=lines['guj_sentence'].apply(lambda x: x.lower())

In [ ]:
# Remove quotes
lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub("'", '', x))
lines['guj_sentence']=lines['guj_sentence'].apply(lambda x: re.sub("'", '', x))

In [ ]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines['english_sentence']=lines['english_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines['guj_sentence']=lines['guj_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [ ]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.translate(remove_digits))
lines['guj_sentence']=lines['guj_sentence'].apply(lambda x: x.translate(remove_digits))
lines['guj_sentence'] = lines['guj_sentence'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

# Remove extra spaces
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.strip())
lines['guj_sentence']=lines['guj_sentence'].apply(lambda x: x.strip())
lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub(" +", " ", x))
lines['guj_sentence']=lines['guj_sentence'].apply(lambda x: re.sub(" +", " ", x))

In [ ]:
lines.head()

,english_sentence,guj_sentence
0,the union cabinet chaired by prime minister shri narendra modi has approved the proposal for closure of autonomous bodies namely rashtriya arogya nidhi ran and jansankhya sthirata kosh jsk and the functions are proposed to be vested in department of health family welfare dohfw,પ્રધાનમંત્રી શ્રી નરેન્દ્ર મોદીની અધ્યક્ષતામાં મંત્રીમંડળે સ્વાસ્થ્ય અને પરિવાર કલ્યાણ વિભાગ અંતર્ગત સ્વાયત્ત સંસ્થાઓને તર્કસંગત બનાવવાની મંજૂરી આપી છે
1,the rationalization of autonomous bodies under department of health family welfare will involve interministerial consultations and review of existing bye laws of these bodies,આ સંસ્થાઓમાં રાષ્ટ્રીય આરોગ્ય નિધિ આરએએન અને જનસંખ્યા સ્થિરતા કોષ જેએસકે સામેલ છે
2,the time frame for implementation is one year,તેનાં કામકાજને સ્વાસ્થ્ય અને પરિવાર કલ્યાણ વિભાગ અંતર્ગત રાખવાનો પ્રસ્તાવ પણ છે
3,rashtriya arogya nidhi ran was set up as a registered society to provide financial medical assistance to poor patients receiving treatment in designated central government hospitals,રાષ્ટ્રીય આરોગ્ય નિધિની રચના એક નોંધાયેલી સોસાયટી સ્વરૂપે કરવામાં આવી હતી જેથી કેન્દ્રીય સરકારી હોસ્પિટલોમાં સારવાર કરાવતાં ગરીબ દર્દીઓને નાણાકીય ચિકિત્સા સહાયતા આપી શકાશે
4,an advance is placed with the medical superintendents of such hospitals who then provide assistance on a case to case basis,આ હોસ્પિટલનાં તબીબી નિરીક્ષકોને આગોતરૂ ભંડોળ આપવામાં આવશે જે દરેક કેસને જોઈને સહાયતા પ્રદાન કરશે


In [ ]:
# Split the data into input and target sentences
input_sentences = []
target_sentences = []
for i in range(len(lines)):
    input_sentences.append(lines['english_sentence'][i])
    target_sentences.append(lines['guj_sentence'][i])

In [ ]:
# Create the vocabulary
input_vocab = sorted(set("".join(input_sentences)))
target_vocab = sorted(set("".join(target_sentences)))

In [ ]:
# Create lookup dictionaries
input_dict = {char: i for i, char in enumerate(input_vocab)}
target_dict = {char: i for i, char in enumerate(target_vocab)}
print(input_dict,"\n",len(input_dict))
print(target_dict,"\n",len(target_dict))

{' ': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, 'अ': 27, 'क': 28, 'ग': 29, 'ण': 30, 'थ': 31, 'न': 32, 'ब': 33, 'म': 34, 'य': 35, 'र': 36, 'ल': 37, 'व': 38, 'ष': 39, 'स': 40, 'ह': 41, 'ा': 42, 'ि': 43, 'ी': 44, 'ृ': 45, 'े': 46, 'ो': 47, 'ौ': 48, '्': 49, '–': 50, '—': 51, '‘': 52, '’': 53, '“': 54, '”': 55, '•': 56} 
 57
{' ': 0, 'a': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'i': 7, 'm': 8, 'n': 9, 'o': 10, 'p': 11, 'r': 12, 's': 13, 't': 14, 'v': 15, 'y': 16, 'ઁ': 17, 'ં': 18, 'ઃ': 19, 'અ': 20, 'આ': 21, 'ઇ': 22, 'ઈ': 23, 'ઉ': 24, 'ઊ': 25, 'ઋ': 26, 'એ': 27, 'ઐ': 28, 'ઑ': 29, 'ઓ': 30, 'ઔ': 31, 'ક': 32, 'ખ': 33, 'ગ': 34, 'ઘ': 35, 'ચ': 36, 'છ': 37, 'જ': 38, 'ઝ': 39, 'ઞ': 40, 'ટ': 41, 'ઠ': 42, 'ડ': 43, 'ઢ': 44, 'ણ': 45, 'ત': 46, 'થ': 47, 'દ': 48, 'ધ': 49, 'ન': 50, 'પ': 51, 'ફ': 52, 'બ': 53, 'ભ': 54, 'મ': 

In [ ]:
# Determine the maximum sequence length
max_len_input = max([len(sentence) for sentence in input_sentences])
max_len_target = max([len(sentence) for sentence in target_sentences])

In [ ]:
# Create the encoder input data
encoder_input_data = np.zeros((len(input_sentences), max_len_input, len(input_vocab)), dtype="float32")
for i, sentence in enumerate(input_sentences):
    for j, char in enumerate(sentence):
        encoder_input_data[i, j, input_dict[char]] = 1.0

In [ ]:
# Create the decoder input data and target data
decoder_input_data = np.zeros((len(target_sentences), max_len_target, len(target_vocab)), dtype="float32")
decoder_target_data = np.zeros((len(target_sentences), max_len_target, len(target_vocab)), dtype="float32")
for i, sentence in enumerate(target_sentences):
    for j, char in enumerate(sentence):
        decoder_input_data[i, j, target_dict[char]] = 1.0
        if j > 0:
            decoder_target_data[i, j - 1, target_dict[char]] = 1.0

In [ ]:
# Step 2: Build the GRU model
encoder_inputs = tf.keras.layers.Input(shape=(None, len(input_vocab)))
encoder = tf.keras.layers.GRU(91, return_state=True)
encoder_outputs, state_h = encoder(encoder_inputs)
encoder_states = [state_h]

decoder_inputs = tf.keras.layers.Input(shape=(None, len(target_vocab)))
decoder_gru = tf.keras.layers.GRU(91, return_sequences=True, return_state=True)
decoder_outputs, _ = decoder_gru(decoder_inputs, initial_state=state_h)
decoder_dense = tf.keras.layers.Dense(len(target_vocab), activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
# Step 3: Train the model
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=1,
    epochs=1,
    validation_split=0.4,
)

600/600 [==============================] - 43s 67ms/step - loss: 0.4623 - val_loss: 0.3973


In [ ]:
# Step 4: Evaluate the model
scores = model.evaluate([encoder_input_data, decoder_input_data], decoder_target_data, verbose=0)
print("Test loss:", scores)

Test loss: 0.4250679314136505


In [ ]:
# Step 5: Use the model for translation
encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)

decoder_state_input = tf.keras.layers.Input(shape=(256,))
decoder_outputs, decoder_state_h = decoder_gru(decoder_inputs, initial_state=decoder_state_input)
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = tf.keras.models.Model([decoder_inputs, decoder_state_input], [decoder_outputs, decoder_state_h])

ValueError: ignored

In [ ]:
def translate(input_sentence):
    # Encode the input sentence
    states_value = encoder_model.predict(np.array([input_sentence]))
    
    # Initialize the target sequence with a start token
    target_seq = np.zeros((1, 1, len(target_vocab)))
    # Generate the target sequence one character at a time
    stop_condition = False
    translation = ""
    while not stop_condition:
        output_tokens, h = decoder_model.predict([target_seq] + states_value)
        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = target_vocab[sampled_token_index]
        # Append the sampled character to the translation
        translation += sampled_char
        # Check if the stop condition is met
        if sampled_char == "\n" or len(translation) > max_len_target:
            stop_condition = True
        # Update the target sequence
        target_seq = np.zeros((1, 1, len(target_vocab)))
        target_seq[0, 0, sampled_token_index] = 1.0
        # Update the states
        states_value = [h]
    
    return translation

In [ ]:
# Test the translation function
input_sentence = encoder_input_data[0]
translation = translate(input_sentence)
print("Input sentence:", input_sentences[0])
print("Translation:", translation[:-1])

1/1 [==============================] - 0s 45ms/step


ValueError: ignored